# Lars

In [1]:
import numpy as np
import pandas as pd
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(30, 30),
            nn.ReLU(),
            nn.Linear(30, 7),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=30, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=7, bias=True)
  )
)


## Daten

In [5]:
df = pd.read_csv("data/train.csv")

In [6]:
df

,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,20753,Male,25.137087,1.766626,114.187096,yes,yes,2.919584,3.000000,Sometimes,no,2.151809,no,1.330519,0.196680,Sometimes,Public_Transportation,Obesity_Type_II
20754,20754,Male,18.000000,1.710000,50.000000,no,yes,3.000000,4.000000,Frequently,no,1.000000,no,2.000000,1.000000,Sometimes,Public_Transportation,Insufficient_Weight
20755,20755,Male,20.101026,1.819557,105.580491,yes,yes,2.407817,3.000000,Sometimes,no,2.000000,no,1.158040,1.198439,no,Public_Transportation,Obesity_Type_II
20756,20756,Male,33.852953,1.700000,83.520113,yes,yes,2.671238,1.971472,Sometimes,no,2.144838,no,0.000000,0.973834,no,Automobile,Overweight_Level_II


In [7]:
df_ohe = pd.get_dummies(df[df.columns.values[:-1]]) # OHE without target column

In [8]:
df_ohe = df_ohe.join(df["NObeyesdad"], how="left")

In [9]:
# Encode target
class_vals = sorted(df_ohe["NObeyesdad"].unique())
class_to_label = {cls: idx for idx, cls in enumerate(class_vals)}

In [10]:
X = torch.tensor(df_ohe.drop(columns=["NObeyesdad", "id"]).astype(float).values
, dtype=torch.float)
y = torch.tensor([class_to_label[class_] for class_ in df_ohe["NObeyesdad"]], dtype=torch.float)

In [11]:
print(X.shape, y.shape)

torch.Size([20758, 30]) torch.Size([20758])


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, # 20% test, 80% train
                                                    random_state=42) # make the random split reproducible


### Untrained Predictions

In [13]:
X_test.shape

torch.Size([4152, 30])

In [14]:
model(X_test.to(device))

tensor([[-8.1301, -1.2012, -1.7834,  ...,  3.1587,  6.3653, -1.9653],
        [-5.8606, -0.8428, -1.3444,  ...,  2.2520,  4.6508, -1.2789],
        [-6.6213, -1.0947, -1.7245,  ...,  2.7535,  4.9125, -1.3070],
        ...,
        [-6.2057, -1.0110, -1.5401,  ...,  2.6531,  4.6736, -1.4792],
        [-7.6366, -1.1980, -2.2856,  ...,  3.4989,  5.3622, -1.5061],
        [-6.1310, -0.9223, -1.5499,  ...,  2.5465,  4.7174, -1.3378]],
       grad_fn=<AddmmBackward0>)

## Training auf Daten

### Loss und Optimizer

In [15]:
loss_fn = nn.CrossEntropyLoss()

In [16]:
optimizer = torch.optim.SGD(params=model.parameters(), 
                            lr=0.1)

In [17]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

### Training

In [18]:
y_logits = model(X_train.to(device))[:1]
y_logits

tensor([[-6.9284, -0.9564, -1.6567,  1.8305,  2.4791,  5.2856, -1.2386]],
       grad_fn=<SliceBackward0>)

In [19]:
y_pred_prob = torch.sigmoid(y_logits)
y_pred_prob

tensor([[9.7857e-04, 2.7760e-01, 1.6021e-01, 8.6182e-01, 9.2266e-01, 9.9496e-01,
         2.2468e-01]], grad_fn=<SigmoidBackward0>)

In [20]:
y_pred = torch.round(y_pred_prob)

print(y_test[:5])
print(y_pred[:5])

tensor([4., 5., 2., 3., 0.])
tensor([[0., 0., 0., 1., 1., 1., 0.]], grad_fn=<SliceBackward0>)


In [21]:
y_logits   

tensor([[-6.9284, -0.9564, -1.6567,  1.8305,  2.4791,  5.2856, -1.2386]],
       grad_fn=<SliceBackward0>)

In [22]:
y_train

tensor([2., 0., 3.,  ..., 5., 4., 2.])

In [23]:
torch.manual_seed(42)

# Set the number of epochs
epochs = 100

# Put data to target device
X_train, y_train = X_train.to(device), y_train.to(device).long()
X_test, y_test = X_test.to(device), y_test.to(device).long()

# Build training and evaluation loop
for epoch in range(epochs):
    ### Training
    model.train()

    # 1. Forward pass (model outputs raw logits)
    y_logits = model(X_train)  # no need to squeeze
    y_pred = torch.argmax(y_logits, dim=1)  # get predicted class

    # 2. Calculate loss/accuracy
    loss = loss_fn(y_logits, y_train)  # no sigmoid needed for CrossEntropyLoss
    acc = accuracy_fn(y_true=y_train, y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_logits = model(X_test)
        test_pred = torch.argmax(test_logits, dim=1)
        # 2. Calculate loss/accuracy
        test_loss = loss_fn(test_logits, y_test)
        test_acc = accuracy_fn(y_true=y_test, y_pred=test_pred)

    # Print out what's happening every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")


Epoch: 0 | Loss: 4.88843, Accuracy: 11.70% | Test loss: 47.67295, Test acc: 19.36%
Epoch: 10 | Loss: 1.95437, Accuracy: 11.78% | Test loss: 1.95390, Test acc: 12.89%
Epoch: 20 | Loss: 1.95288, Accuracy: 12.09% | Test loss: 1.95151, Test acc: 12.38%
Epoch: 30 | Loss: 1.94717, Accuracy: 19.52% | Test loss: 1.94627, Test acc: 19.36%
Epoch: 40 | Loss: 1.94293, Accuracy: 19.52% | Test loss: 1.94241, Test acc: 19.36%
Epoch: 50 | Loss: 1.93977, Accuracy: 19.52% | Test loss: 1.93957, Test acc: 19.36%
Epoch: 60 | Loss: 1.93742, Accuracy: 19.52% | Test loss: 1.93748, Test acc: 19.36%
Epoch: 70 | Loss: 1.93568, Accuracy: 19.52% | Test loss: 1.93595, Test acc: 19.36%
Epoch: 80 | Loss: 1.93437, Accuracy: 19.52% | Test loss: 1.93483, Test acc: 19.36%
Epoch: 90 | Loss: 1.93340, Accuracy: 19.52% | Test loss: 1.93401, Test acc: 19.36%
